In [1]:
import numpy as np
import pandas as pd
from io import BytesIO,StringIO
from nanoHUB.application import Application
from nanoHUB.pipeline.geddes.data import get_default_s3_client


application = Application.get_instance()
salesforce = application.new_salesforce_engine()

s3_client = get_default_s3_client(application)

nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [2]:
contacts_in_cluster_df = salesforce.query_data('Select Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')
contacts_df = salesforce.query_data('Select Id, nanoHUB_user_ID__c from Contact')

self_study = []
clustered = []

sf_contact_ids = contacts_df['Id'].to_list()

unique_contacts_in_clusters = set(contacts_in_cluster_df['Contact__c'].to_list())
bool_search = np.in1d(np.array(sf_contact_ids),np.array(list(unique_contacts_in_clusters)))

clustered = np.where(bool_search)[0].tolist()
self_study = np.where(~bool_search)[0].tolist()

clustered_NH_ids = contacts_df['nanoHUB_user_ID__c'][clustered]
self_study_NH_ids = contacts_df['nanoHUB_user_ID__c'][self_study]


[Success] Bulk job creation successful. Job ID = 7505w00000cxBUbAAM
{"id":"7505w00000cxBUbAAM","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:26:50.000+0000","systemModstamp":"2022-01-19T15:26:50.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000cxBUbAAM","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:26:50.000+0000","systemModstamp":"2022-01-19T15:26:57.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":98068,"retries":0,"totalProcessingTime":5164}
[Success] Bulk job completed successfully.
[Success] Bulk job creation succes

In [3]:
self_study_NH_df = self_study_NH_ids.to_frame()
display(self_study_NH_df)


,nanoHUB_user_ID__c
0,998.0
1,1683.0
2,1684.0
3,1685.0
4,1686.0
...,...
251331,344663.0
251332,344664.0
251333,344666.0
251334,344667.0


In [4]:
salesforce_query = '''
SELECT Id, nanoHUB_user_ID__c 
FROM Contact
    WHERE Id NOT IN (Select Contact__c from ContactToolClusterAssociation__c)
'''

In [5]:
result = salesforce.query_data(salesforce_query)
display(result)

[Success] Bulk job creation successful. Job ID = 7505w00000cxBUhAAM
{"id":"7505w00000cxBUhAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:27:28.000+0000","systemModstamp":"2022-01-19T15:27:28.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000cxBUhAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:27:28.000+0000","systemModstamp":"2022-01-19T15:27:28.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":187860,"retries":0,"totalProcessingTime":10166}
{"id":"7505w00000cxBUhAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","c

,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w00003WP7uUAAT,341493.0
2,0035w000034IQUpAAO,155491.0
3,0035w000035jWJkAAM,287673.0
4,0035w000034JJn7AAG,28717.0
...,...,...
187855,0035w000034I4w9AAC,82105.0
187856,0035w000034JoNfAAK,113556.0
187857,0035w000034JMFlAAO,30987.0
187858,0035w000034Jz4UAAS,205903.0


In [6]:
display(result.nanoHUB_user_ID__c.nunique())

187842

In [7]:
intersection = pd.merge(result, self_study_NH_ids, how='inner', on=['nanoHUB_user_ID__c'])

In [8]:
display(intersection.nunique())

Id                    187860
nanoHUB_user_ID__c    187842
dtype: int64

In [9]:
difference1 = result.merge(
    self_study_NH_df, on='nanoHUB_user_ID__c', how='outer', suffixes=['', '_'], indicator=True
).loc[lambda x : x['_merge']!='both']

display(difference1)

,Id,nanoHUB_user_ID__c,_merge


In [10]:
from nanoHUB.repositories import UnclassifiedUsers, ClassroomUsers

In [11]:
repo = UnclassifiedUsers(salesforce)
unclassified_users_df = repo.get_all()
display(unclassified_users_df)

[Success] Bulk job creation successful. Job ID = 7505w00000cxBUlAAM
{"id":"7505w00000cxBUlAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:27:55.000+0000","systemModstamp":"2022-01-19T15:27:55.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000cxBUlAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:27:55.000+0000","systemModstamp":"2022-01-19T15:27:55.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":19276,"retries":0,"totalProcessingTime":1471}
{"id":"7505w00000cxBUlAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T1

,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w00003WP7uUAAT,341493.0
2,0035w000034IQUpAAO,155491.0
3,0035w000035jWJkAAM,287673.0
4,0035w000034JJn7AAG,28717.0
...,...,...
187855,0035w000034I4w9AAC,82105.0
187856,0035w000034JoNfAAK,113556.0
187857,0035w000034JMFlAAO,30987.0
187858,0035w000034Jz4UAAS,205903.0


In [13]:
full_path = 'unclassified_users.parquet.gzip' 
bucket_name = 'nanohub.processed'



_buf = BytesIO()
unclassified_users_df.to_parquet(_buf, index=False)
_buf.seek(0)
s3_client.put_object(Bucket=bucket_name, Body=_buf.getvalue(), Key=full_path)

{'ResponseMetadata': {'RequestId': 'tx0000000000000000234b0-0061e82e16-e068dd-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Jan 2022 15:28:22 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'etag': '"21dbb612eb85717b791310eaf73bcf9b"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx0000000000000000234b0-0061e82e16-e068dd-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'ETag': '"21dbb612eb85717b791310eaf73bcf9b"'}

In [14]:
repo = ClassroomUsers(salesforce)
classroom_users_df = repo.get_all()
display(classroom_users_df)

full_path = 'classroom_users.parquet.gzip' 
bucket_name = 'nanohub.processed'

_buf = BytesIO()
classroom_users_df.to_parquet(_buf, index=False)
_buf.seek(0)
s3_client.put_object(Bucket=bucket_name, Body=_buf.getvalue(), Key=full_path)

[Success] Bulk job creation successful. Job ID = 7505w00000cxBUqAAM
{"id":"7505w00000cxBUqAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:28:22.000+0000","systemModstamp":"2022-01-19T15:28:22.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000cxBUqAAM","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-19T15:28:22.000+0000","systemModstamp":"2022-01-19T15:28:27.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":63476,"retries":0,"totalProcessingTime":3835}
[Success] Bulk job completed successfully.


,Id,nanoHUB_user_ID__c
0,0035w000034Jo08AAC,125212.0
1,0035w000034IrzvAAC,276235.0
2,0035w000034Is2GAAS,276281.0
3,0035w000034IXGEAA4,176759.0
4,0035w000034JtxGAAS,98366.0
...,...,...
63471,0035w000034IWuSAAW,166199.0
63472,0035w000034IWuOAAW,166195.0
63473,0035w000034IWuKAAW,166188.0
63474,0035w000034IX2eAAG,166411.0


{'ResponseMetadata': {'RequestId': 'tx0000000000000000234b2-0061e82e23-e068dd-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Jan 2022 15:28:35 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'etag': '"9639932b20bc4fbc734f937b14ba3247"',
   'accept-ranges': 'bytes',
   'x-amz-request-id': 'tx0000000000000000234b2-0061e82e23-e068dd-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'ETag': '"9639932b20bc4fbc734f937b14ba3247"'}

In [15]:
for key in s3_client.list_objects(Bucket='nanohub.processed')['Contents']:
    obj = s3_client.get_object(Bucket='nanohub.processed', Key=key['Key'])
    print(key['Key'])
    if key['Key'].endswith('.parquet.gzip'):
        display(pd.read_parquet(BytesIO(obj['Body'].read())))
    else:
        display(pd.read_csv(BytesIO(obj['Body'].read())))
    

classroom_users.parquet.gzip


,Id,nanoHUB_user_ID__c
0,0035w000034Jo08AAC,125212.0
1,0035w000034IrzvAAC,276235.0
2,0035w000034Is2GAAS,276281.0
3,0035w000034IXGEAA4,176759.0
4,0035w000034JtxGAAS,98366.0
...,...,...
63471,0035w000034IWuSAAW,166199.0
63472,0035w000034IWuOAAW,166195.0
63473,0035w000034IWuKAAW,166188.0
63474,0035w000034IX2eAAG,166411.0


core_cost_clustered_users.csv


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,njohnson463,txu,ltong,svs470,eko,iezzoni21,gwm166,bmaganti,gpedrick,bsh319,...,sfedorikhina,dchun,asklar,cmpark,relwood,gthippeswamy,mayabhardwaj,jglass,noheneadu,dianemok
1,svpeders,clyons,bphester1,lbehrens,bluegreentwo,alimiadi,alentner12,mille273,mokomi89,mwjensen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ee21yin,zchng2,esunny2,chenchen,rbalik,tesler2,sramin3,nhamer2,bridgwater,mstephe2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vmanane,mgsassenfeld,aelgkorv,jmmontes2,manaya,dmuse,danishr,champinhon,yvette,daesparza3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hkb,nizan123,haika,Tal,sduppleganger,shogun,guy23,itaikam,baraklit,hila,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,mhossai,ishsman,kjs1000,gu0,nihaoya,laplaceiitkgp,cchong,yoster84,ssuslov,pree2cool,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,torbentage,bjarkemalm,bluewolfrulz,jnunez,laur,robertpennington,nori86,marsmand,mhca,anderssig,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,kingya,sheng,silverjbj,pythais,benny7,spiderweb432,karlwr,jtli3,jazzphysicist,bumjin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,vkemper,ch_klar,alleinherrscher,thommy,masa,mkes1982,khalsa,mwidera,nilsb,zaradras,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,swetha443,mahi2234,nanotechnology530,jaejoon,ep4,mushtaq0455,subbu,spulli,praveen33,desidudes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cost_cluster_analysis/by_semester/2008-01-01_2008-07-01.csv


...

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,aliffizham,bornaobradovic,renyi0912,iron52bat,yinyong0427,shafiq__,faizyus07,chinkeatchew880,kiranjot.kaur.3388,um17206869,...,crce.8752.elec,aqbarshaiful,maxcleveland73,shahrizan01,tangkityee,chiraghegde101,iskandarsofian,alonb96,u2003990,chiachenng123
1,e.silviagarcia12,mparody,mich_loiz,luimanesp,claracd,duenas5a,cobos,setarcos,jose.lop.mol98,ivancamacho4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gagnonn,irwin12,voorsm,sharlow,bbarale,bwodetz,gszyman,tdeucher,kradde,tschoste,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,arodriguez30,pr4b0w0,a01632679,connorduffy,sophia.corrales,anna.langley,walter.sanchez,walkert1,cami,dyuna.soledade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,panellink1,caseyamick,u1109841,hlong262,edakartari,ege.ert7,halilalp,sgnmsrmsk,yusufsabahoglu05,elel30182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,irfanalfarizi07,asd12amien,fariz,riazoldik,putrinurfianana,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,dctbelieve,duswl2510,yoonwe00,seunghun4757,hilsong,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,ee20m540,vandermeulen.niels,djuncool,nitze.velascomartinez,evetec,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,grsatheeshk,p.254143,matt,antonio.chaves,kamthekarg,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


cost_cluster_analysis/by_semester/2021-07-02_2021-12-31.csv


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,shah.di,vpb,pranavsachdeva2002,aniket.ubhare,saikalyan2002,kushmundra,shailendramaurya,manunoah,ananya,avi.amalanshu,...,krishnchate121,shubham1401,rahulkr2038,smita.nayak,arnavaryaman,aniket.mps,shrutisharma,barathbommadi,marioxpeach,yashas_more
1,genzo,iady_parodi,giorgio17,ildottore01,silviacostanza.sc,dimitricresp,alessio.montanaro,edoardo02,lorynova01,simone12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,maoh2,akingery,juliahd2,sali93,sofiamv2,rsears4,lasingek,sopfeil2,jiming2,daxia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sandeep.patel,chco7711,ojas12,adithya.20ec007,manasa.20ec028,serdarsallanti43,ogu0001,seifallh,sankalp.20ec052,cetinkayaahmetcan17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,juliannemaevergara4,appoluz.almario.pharma,yjzhuang5,zammarianne.gutierrez.pharma,elielfrances.toledo.pharma,jeblgd.ph,raizamaureen.matias.pharma,jhudielacupanda,vincent.villanueva.pharma,diannejoyce.saragpon.pharma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,rsoniya664,gioia.chiodi95,bhavishyapelluru19,sravanthiduggu09,sathishv18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,carloseduardorufino7,toptap638,wizard.justus,floriandobreff24,maar4788,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,jyy2138,jm5326,vp2463,yc3254,lam2286,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,benpitblado18,rihand,sdhamank,taeksanglee,igi.chladek,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


unclassified_users.parquet.gzip


,Id,nanoHUB_user_ID__c
0,0035w000034IS8EAAW,158298.0
1,0035w00003WP7uUAAT,341493.0
2,0035w000034IQUpAAO,155491.0
3,0035w000035jWJkAAM,287673.0
4,0035w000034JJn7AAG,28717.0
...,...,...
187855,0035w000034I4w9AAC,82105.0
187856,0035w000034JoNfAAK,113556.0
187857,0035w000034JMFlAAO,30987.0
187858,0035w000034Jz4UAAS,205903.0
